In [4]:
import yfinance as yf
import pandas as pd

# Define tickers and time period
tickers = ['SPY', 'AAPL']
start_date = '2010-01-01'
end_date = '2025-09-30'

# Download data
data = yf.download(tickers, start=start_date, end=end_date)

if isinstance(data.columns, pd.MultiIndex):
    adj_close = data['Close']                  # adjusted closes
else:
    adj_close = data[['Close']].rename(columns={'Close': tickers[0]})


/var/folders/9z/26jz_ql9507379s_bv_2qyq40000gn/T/ipykernel_81973/1666811093.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  2 of 2 completed


In [15]:
import yfinance as yf
import pandas as pd
import numpy as np

def download_ohlc(tickers, start='2010-01-01', end=None):
    data = yf.download(list(tickers), start=start, end=end, auto_adjust=True, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        out = {tk: data.xs(tk, axis=1, level=1)[['Open','High','Low','Close']].dropna()
               for tk in tickers}
    else:  # single ticker
        tk = tickers[0] if isinstance(tickers, (list, tuple)) else tickers
        out = {tk: data[['Open','High','Low','Close']].dropna()}
    return out

def tally_trailing_triggers(df, thresholds=(0.05, 0.10, 0.15)):
    c = df['Close']
    roll_max = c.cummax()
    dd = c / roll_max - 1.0  # negative when below the running high

    rows = []
    dates_map = {}
    for t in thresholds:
        crossed = (dd <= -t) & (dd.shift(1) > -t)  # first day it breaches that level
        dates = list(dd.index[crossed])
        rows.append({'Threshold %': int(t*100), '# Triggers': len(dates)})
        dates_map[int(t*100)] = dates
    return pd.DataFrame(rows), dates_map

def tally_fixed_from_entry(df, thresholds=(0.05, 0.10, 0.15)):
    c = df['Close']
    entry = c.iloc[0]
    stops = {int(t*100): entry * (1 - t) for t in thresholds}
    out = []
    dates_map = {}
    for pct, stop_level in stops.items():
        hit = (df['Low'] <= stop_level)
        if hit.any():
            first_date = df.index[hit.values.argmax()]  # first True
            out.append({'Threshold %': pct, '# Triggers': 1})
            dates_map[pct] = [first_date]
        else:
            out.append({'Threshold %': pct, '# Triggers': 0})
            dates_map[pct] = []
    return pd.DataFrame(out), dates_map

def summarize_tallies(tickers=('SPY','AAPL'), start='2010-01-01', mode='trailing'):
    data = download_ohlc(tickers, start=start)
    rows = []
    dates_by_ticker = {}
    for tk, df in data.items():
        if mode == 'trailing':
            tbl, dates = tally_trailing_triggers(df, thresholds=(0.05,0.10,0.15))
        elif mode == 'fixed':
            tbl, dates = tally_fixed_from_entry(df, thresholds=(0.05,0.10,0.15))
        else:
            raise ValueError("mode must be 'trailing' or 'fixed'")
        tbl.insert(0, 'Ticker', tk)
        rows.append(tbl)
        dates_by_ticker[tk] = dates
    summary = pd.concat(rows, ignore_index=True)
    return summary, dates_by_ticker



    

In [16]:
summary_trailing, dates_trailing = summarize_tallies(('SPY','AAPL'), start='2010-01-01', mode='trailing')
print(summary_trailing); dates_trailing['SPY'][5]  # list of dates when 5% trailing stop was triggered

  Ticker  Threshold %  # Triggers
0    SPY            5          65
1    SPY           10          37
2    SPY           15          29
3   AAPL            5         114
4   AAPL           10          85
5   AAPL           15          51


[Timestamp('2010-01-22 00:00:00'),
 Timestamp('2010-01-28 00:00:00'),
 Timestamp('2010-02-04 00:00:00'),
 Timestamp('2010-05-06 00:00:00'),
 Timestamp('2010-05-14 00:00:00'),
 Timestamp('2010-09-27 00:00:00'),
 Timestamp('2010-09-29 00:00:00'),
 Timestamp('2010-10-04 00:00:00'),
 Timestamp('2011-03-16 00:00:00'),
 Timestamp('2011-06-06 00:00:00'),
 Timestamp('2011-06-22 00:00:00'),
 Timestamp('2011-08-01 00:00:00'),
 Timestamp('2011-10-31 00:00:00'),
 Timestamp('2012-05-14 00:00:00'),
 Timestamp('2012-06-21 00:00:00'),
 Timestamp('2012-07-12 00:00:00'),
 Timestamp('2012-07-24 00:00:00'),
 Timestamp('2012-11-08 00:00:00'),
 Timestamp('2013-06-24 00:00:00'),
 Timestamp('2014-02-03 00:00:00'),
 Timestamp('2014-10-10 00:00:00'),
 Timestamp('2015-08-21 00:00:00'),
 Timestamp('2015-10-13 00:00:00'),
 Timestamp('2016-01-06 00:00:00'),
 Timestamp('2016-03-08 00:00:00'),
 Timestamp('2016-06-27 00:00:00'),
 Timestamp('2018-02-05 00:00:00'),
 Timestamp('2018-02-20 00:00:00'),
 Timestamp('2018-02-